In [ ]:
import nltk
import Query_generation as Qg
import QuestionClassifier as QC
from nltk.tokenize import sent_tokenize
from nltk.chunk import *
import operator
from nltk.corpus import stopwords
import math as m
from fractions import Fraction
import re

In [ ]:
Document_set = Qg.Document_subset(Qg.ans,20)
stop = set(stopwords.words('english'))

In [ ]:
def Rule_Based_AnswerSystem(Question):
    Question_class = QC.test_questions(Question)
    rule = {'HUM':'PERSON','LOC':'GPE','ENTY':['NNP','NN','NNS'],'DESC':['NNP','NN','NNS'],'NUM':'CD','ABBR':['NNP','NN','NNS']}
    Ans_class = rule[Question_class]
    
    return Ans_class

In [ ]:
def filterAnsByLabel(Document_set,Ans_class):
    Ans_Back_up = []
    for Document in Document_set:
        file = open(Document)
        for row in file:
            row = row.replace('\n','')
            row = row.replace('\r','')
            try:
                sentence = sent_tokenize(row)
            except:
                continue
            
            for singleSentence in sentence:
                flag = False
                words = singleSentence.split()
                for word in words:
                    if Query_term.has_key(word) and flag == False:
                        Ans_Back_up.append(singleSentence)
                        flag = True
                        
    final_ans = []
    for sentence in Ans_Back_up:
        flag = False
        token = nltk.word_tokenize(sentence)
        pos_tags = nltk.pos_tag(token)
        result_ = nltk.ne_chunk(pos_tags,binary = False)
        
        for item in result_:
            try:
                if item.label() == Ans_class and flag == False:
                    final_ans.append(sentence)
                    flag = True
                    
            except:
                if flag == False:
                    if len(Ans_class) == 3:
                        inerflag = False
                        for tag in pos_tags:
                            if tag[1] == 'CD':
                                inerflag = True
                        if inerflag == False:
                            final_ans.append(sentence)
                            flag = True
                    elif Ans_class == 'CD':
                        if item[1] == 'CD':
                            final_ans.append(sentence)
                            flag = True
    return final_ans

In [ ]:
def bigrams(sentence):
    stop = set(stopwords.words('english'))
    Q = nltk.word_tokenize(sentence)
    result = []
    result.extend([i for i in Q if i not in stop])
    result_bigram = list(nltk.bigrams(result))

In [ ]:
def Answer_Score_System(Ans_set,question_):
    question = nltk.word_tokenize(question_)
    pattern = r'[^A-Za-z0-9\s]+'
    question = re.sub(pattern,'',question_)
    question_bigram = list(nltk.bigrams(question))
    Number_of_Same_word = 0
    total_score = {}
    for sentence in Ans_set:
        label = sentence
        score = 0
        sentence = nltk.word_tokenize(sentence)
        sentence_bigram = list(nltk.bigrams(sentence))
        #score of bigram match
        for bigram in question_bigram:
            for item in sentence_bigram:
                if item == bigram:
                    score = score + 2
        #score of unigram match
        for item in question:
            for word in sentence:
                if word == item:
                    score = score + 1
                    
        score = Fraction(score,len)

In [ ]:
def TF_TDF(keywords, Relevant_Documents, total_Counts):
    qtf = keywords
    df = Doc_Freq(keywords, Relevant_Documents)
    tf_lists = {}
    for document in Relevant_Documents:
        tf = Term_Freq(keywords, document)
        if tf != {}:
            tf_lists[document] = tf
            
    TF_IDF = {}
    for key, val in tf_lists.iterirems():
        score = 0
        term = val
        max_term = MaxTerm(key)
        for eachterm, value in term.iteritems():
            TF = 0.5 + (0.5 * value / max_term)
            IDF = m.log(Fraction(total_Counts, df[eachterm]))
            
            tf_idf = TF * IDF
            score = score + tf_idf
            
        if score != 0:
            TF_IDF[key] = score
            
    return sorted(TF_IDF.iteritems(), key = operator.itemgetter(1), reverse = True)


In [ ]:
def Term_Select(question):
    stop = set(stopwords.word('english'))
    regex = r'[^A-Za-z0-9\s]+'
    question = re.sub(regex, '', question)
    tokenwords = nltk.word_tokenize(question)
    lists = []
    for word in tokenwords:
        if word not in stop:
            lists.append(word)
            
    pos_tags = nltk.pos_tag(lists)
    Query_Terms = {}
    for item in pos_tags:
        if item[0] != 'What' and item[0] != 'Which' and item[0] != 'When' and item[0] != 'Where' and item[0] != 'Whose' and item[0] != 'Who' and item[0] != 'How' and not Query_Terms.has_key(item[0]):
            Query_Terms[item[0]] = 1
        elif item[0] != 'What' and item[0] != 'Which' and item[0] != 'When' and item[0] != 'Where' and item[0] != 'Whose' and item[0] != 'Who' and item[0] != 'How' and Query_Terms.has_key(item[0]):
            Query_Terms[item[0]] = Query_Terms[item[0]] + 1
            
    return Query_Terms

In [ ]:
def MaxTerm(Document):
    file = open(Document)
    MaximumTerms = {}
    stop = set(stopwords.words('English'))
    for row in file:
        row = row.replace('\n','')
        words = row.split(' ')
        for word in words:
            if word not in stop and word != 'I' and word != 'The' and word != 'would' and word != 'Would':
                if not MaximumTerms.has_key(word):
                    MaximumTerms[word] = 1
                else:
                    MaximumTerms[word] = MaximumTerms[word] + 1
    return max(MaximumTerms.iteritems(), key=operator.itemgetter(1))[1]

In [ ]:
def Term_frequency(queryterm,sentence):
    tf = dict.fromkeys(queryterm.keys(),0)
    for word in sentence:
        if queryterm.has_key(word):
            tf[word] = tf[word] + 1
    return tf

In [ ]:
def question_Term(question):
    question = question.replace('?','')
    question_ = nltk.word_tokenize(question)
    pos_tags = nltk.pos_tag(question_)
    question = []
    for item in pos_tags:
        if item[0] != 'Which':
            if item[1] == 'NNP' or item[1] == 'NNS' or item[1] == 'NN' or item[1] == 'CD' or item[1] == 'JJ':
                question.append(item[0])

    question_term = {}
#create question term

    for item in question:

        if not question_term.has_key(item):
            question_term[item] = 1

        else:

            question_term[item] = question_term[item] + 1

    return question_term 

In [ ]:
Query_term = dict.fromkeys(Qg.queryterms.keys(),0)
Ans_type = Rule_Based_AnswerSystem(Qg.question)

ansset = filterAnsByLabel(Document_set,Ans_type)
# ansset = Document_set
# print Answer_Score_System(ansset,Qg.question)
ans = TF_IDF(ansset,Qg.question)
ans_sub = Document_subset(ans,10)
print ans_sub